#**2장. 신경망의 수학적 구성 요소**

###신경망의 엔진: 그레이디언트 기반 최적화

#####**텐서플로의 그레이디언트 테이프**

In [ ]:
import tensorflow as tf

x = tf.Variable(0.) # tf.Variable은 변경 가능한 상태를 담기 위한 특별한 텐서
with tf.GradientTape() as tape: # GradientTape로 자동 미분 기능을 이용할 수 있음
  y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)

In [ ]:
x = tf.Variable(tf.zeros((2,2)))
with tf.GradientTape() as tape:
  y = 2 * x + 3
grad_of_y_wrt_x = tape.gradient(y, x)

In [ ]:
W = tf.Variable(tf.random.uniform((2, 2)))
b = tf.Variable(tf.zeros((2,)))
x = tf.random.uniform((2,2))
with tf.GradientTape() as tape:
  y = tf.matmul(x, W) + b
grad_of_y_wrt_W_and_b = tape.gradient(y, [W, b])

###첫 번째 예제 다시 살펴보기

#####**첫 번째 예제 다시 살펴보기**

In [ ]:
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([ # Sequential > 레이어들을 하나로 묶어줌
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(optimizer="rmsprop", # compile > training에 필요한 요소들을 하나로 묶어 define 해줌
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

In [ ]:
model.fit(train_images, train_labels, epochs=5, batch_size=128)

Epoch 1/5
469/469 [==============================] - 6s 10ms/step - loss: 0.2669 - accuracy: 0.9235
Epoch 2/5
469/469 [==============================] - 5s 10ms/step - loss: 0.1084 - accuracy: 0.9683
Epoch 3/5
469/469 [==============================] - 6s 12ms/step - loss: 0.0704 - accuracy: 0.9788
Epoch 4/5
469/469 [==============================] - 5s 10ms/step - loss: 0.0517 - accuracy: 0.9841
Epoch 5/5
469/469 [==============================] - 6s 13ms/step - loss: 0.0387 - accuracy: 0.9884


#####**텐서플로를 사용하여 첫 번째 예제를 밑바닥부터 다시 구현하기**

In [ ]:
import tensorflow as flow

class NaiveDense:
  def __init__(self, input_size, output_size, activation):

    self.activation = activation

    w_shape = (input_size, output_size) # (input_size, output_size) 크기의 행렬 w 생성

    w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1) # 요소들을 랜덤값으로 초기화
    self.W = tf.Variable(w_initial_value)

    b_shape = (output_size,) # (output_size,) 크기의 행렬 b 생성
    b_initial_value = tf.zeros(b_shape) # 요소들을 0으로 초기화
    self.b = tf.Variable(b_initial_value)

  def __call__(self, inputs): # 입력부터 출력까지 각 계층의 가중치를 계산해주는 정방향 패스 진행
    return self.activation(tf.matmul(inputs, self.W) + self.b)

  @property
  def weights(self): # 가중치를 return 해줌
    return [self.W, self.b]

In [ ]:
class NaiveSequential:
  def __init__(self, layers):
    self.layers = layers

  def __call__(self, inputs):
    x = inputs
    for layer in self.layers: # 층을 순서대로 호출
      x = layer(x)
    return x

  @property
  def weights(self): # 가중치를 return 해줌
    weights = []
    for layer in self.layers:
      weights += layer.weights
    return weights

In [ ]:
model = NaiveSequential([ # NaiveSequential 클래스를 이용하여 유사 모델 생성
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])

assert len(model.weights) == 4

#####**배치 제너레이터**

In [ ]:
import math

class BatchGenerator:
  def __init__(self, images, labels, batch_size=128):
    assert len(images) == len(labels)
    self.index = 0
    self.images = images
    self.labels = labels
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)

  def next(self):
    images = self.images[self.index : self.index + self.batch_size]
    labels = self.labels[self.index : self.index + self.batch_size]
    self.index += self.batch_size
    return images, labels

#####**훈련 스텝 실행하기**

In [ ]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape: # GradientTape 블록 내에서 정방향 패스를 통해 모델의 예측 계산
      predictions = model(images_batch)
      per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
          labels_batch, predictions)
      average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights) # model.weights에 대해 average_loss의 그레이디언트를 계산
    update_weights(gradients, model.weights) # 그레이디언트를 통한 가중치 업데이트
    return average_loss

learning_rate = 1e-3

def update_weights(gradients, weights):
  for g, w in zip(gradients, weights):
    w.assign_sub(g * learning_rate) # 각 가중치에서 gradients * learning_rate를 빼주어 가중치를 업데이트

In [ ]:
from tensorflow.keras import optimizers

# (윗 블럭의 learning_rate ~ update_weights 정의는 아래 코드와 같음)

optimizer = optimizers.SGD(learning_rate = 1e-3)

def update_weights(gradients, weights): # 윗 블럭처럼 가중치 업데이트는 수동으로 구현하지 않고 지금처럼 optimizer를 이용해 자동으로 함
  optimizer.apply_gradients(zip(gradients, weights))

#####**전체 훈련 루프**

In [ ]:
def fit(model, images, labels, epochs, batch_size=120):
  for epoch_counter in range(epochs): # 각 에포크를 반복함
    print(f"에포크 {epoch_counter}")
    batch_generator = BatchGenerator(images, labels)
    for batch_counter in range(batch_generator.num_batches): # 각 에포크마다 훈련 데이터의 배치에 대한 훈련 스텝 반복
      images_batch, labels_batch = batch_generator.next()
      loss = one_training_step(model, images_batch, labels_batch)
      if batch_counter % 100 == 0:
        print(f"{batch_counter}번째 배치 손실: {loss:.2f}")

In [ ]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128) # 위에서 쭉 정의한 함수들을 이용해 훈련 진행

에포크 0
0번째 배치 손실: 4.97
100번째 배치 손실: 2.21
200번째 배치 손실: 2.18
300번째 배치 손실: 2.07
400번째 배치 손실: 2.21
에포크 1
0번째 배치 손실: 1.91
100번째 배치 손실: 1.86
200번째 배치 손실: 1.81
300번째 배치 손실: 1.71
400번째 배치 손실: 1.83
에포크 2
0번째 배치 손실: 1.59
100번째 배치 손실: 1.56
200번째 배치 손실: 1.48
300번째 배치 손실: 1.41
400번째 배치 손실: 1.51
에포크 3
0번째 배치 손실: 1.33
100번째 배치 손실: 1.33
200번째 배치 손실: 1.22
300번째 배치 손실: 1.20
400번째 배치 손실: 1.28
에포크 4
0번째 배치 손실: 1.14
100번째 배치 손실: 1.15
200번째 배치 손실: 1.02
300번째 배치 손실: 1.04
400번째 배치 손실: 1.11
에포크 5
0번째 배치 손실: 0.99
100번째 배치 손실: 1.01
200번째 배치 손실: 0.89
300번째 배치 손실: 0.92
400번째 배치 손실: 0.98
에포크 6
0번째 배치 손실: 0.88
100번째 배치 손실: 0.91
200번째 배치 손실: 0.79
300번째 배치 손실: 0.83
400번째 배치 손실: 0.90
에포크 7
0번째 배치 손실: 0.80
100번째 배치 손실: 0.83
200번째 배치 손실: 0.71
300번째 배치 손실: 0.77
400번째 배치 손실: 0.83
에포크 8
0번째 배치 손실: 0.73
100번째 배치 손실: 0.76
200번째 배치 손실: 0.65
300번째 배치 손실: 0.71
400번째 배치 손실: 0.78
에포크 9
0번째 배치 손실: 0.68
100번째 배치 손실: 0.71
200번째 배치 손실: 0.60
300번째 배치 손실: 0.67
400번째 배치 손실: 0.74


#####**모델 평가하기**

In [ ]:
import numpy as np

predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"정확도: {matches.mean(): .2f}")

정확도:  0.82
